In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [2]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [3]:
# -------------1 删除重复值------------------

In [4]:
# DataFrame方法duplicated返回的是一个boolean Series，表示一个row是否是重复的（根据前一行来判断）：
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [5]:
#drop_duplicateds返回一个DataFrame，会删除重复的部分：
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [6]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [7]:
data.drop_duplicates(['k1']) #都默认考虑所有列；另外，我们可以指定一部分来检测重复值。假设我们只想检测'k1'列的重复值：

,k1,k2,v1
0,one,1,0
1,two,1,1


In [8]:
# duplicated和drop_duplicated默认保留第一次观测到的数值组合。设置keep='last'能返回最后一个： 
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


In [9]:
# --------------------2. 用函数和映射来转换数据-----------------

In [10]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [11]:
# 假设你想加一列，表明每种肉来源的动物是什么。我们可以写一个映射：map
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}
#用于series的map方法接受一个函数，或是一个字典，包含着映射关系，
#但这里有一个小问题，有些肉是大写，有些是小写。因此，我们先用str.lower把所有的值变为小写:
lowercased = data['food'].str.lower()
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [12]:
# 也可以用一个函数解决上面的问题：
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [13]:
# ------------------3 Replacing Values（替换值）-------------------

In [14]:
# fillna是一个特殊换的替换操作
# map可以用于修改一个object里的部分值，
# 但是replace能提供一个更简单和更灵活的方法做到这点
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [15]:
# -999可能是用来表示缺失值的标识符。用NA来替代的话，用replace，会产生一个新series（除非使用inplace=True）
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [16]:
# 如果想要一次替换多个值，直接用一个list即可：
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [17]:
# 不同的值用不同的替换值，也是导入一个list
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [18]:
# ------------------4 Renaming Axis Indexes（重命名Axis Indexes）----------------------

In [19]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [20]:
transform = lambda x: x[:4].upper()
data.index.map(transform) # 轴标签也有一个map方法

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [21]:
# 如果你想要创建一个转换后的版本，而且不用修改原始的数据，可以用rename:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [22]:
# ------------------5 Discretization and Binning（离散化和装箱）--------------------

In [23]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
# 分到四个bin里，19~25, 26~35, 36~60, >60。可以用pandas里的cut
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
# 返回的是一个特殊的Categorical object
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [25]:
# label标号（可以通过codes属性查看
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [26]:
# categories数组，用来记录不同类别的名字
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [27]:
# pandas.cut后bin的数量
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [28]:
# 这里我们注意一下区间。括号表示不包含，方括号表示包含。你可以自己设定哪一边关闭（right=False）:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)


[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [29]:
# 可以用一个list或数组给labels选项来设定bin的名字
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [30]:
# bins = 数量 ，而不是边界
# 则会均匀切分，(max-min) / bins
data = np.random.rand(20)
pd.cut(data, 4, precision=2)  # precision=2选项表示精确到小数点后两位。

[(0.75, 1.0], (0.26, 0.51], (0.018, 0.26], (0.018, 0.26], (0.018, 0.26], ..., (0.51, 0.75], (0.26, 0.51], (0.26, 0.51], (0.26, 0.51], (0.26, 0.51]]
Length: 20
Categories (4, interval[float64]): [(0.018, 0.26] < (0.26, 0.51] < (0.51, 0.75] < (0.75, 1.0]]

In [31]:
# qcut，会按照数据的分位数来分箱
# 而qcut是按百分比来切的，所以可以得到等数量的bins
data = np.random.randn(1000) # Normally distributed
cats = pd.qcut(data, 4) # Cut into quartiles
cats

[(-3.865, -0.688], (-0.688, 0.0759], (-3.865, -0.688], (-0.688, 0.0759], (0.652, 3.188], ..., (-3.865, -0.688], (0.0759, 0.652], (0.652, 3.188], (0.0759, 0.652], (-3.865, -0.688]]
Length: 1000
Categories (4, interval[float64]): [(-3.865, -0.688] < (-0.688, 0.0759] < (0.0759, 0.652] < (0.652, 3.188]]

In [32]:
pd.value_counts(cats)

(0.652, 3.188]      250
(0.0759, 0.652]     250
(-0.688, 0.0759]    250
(-3.865, -0.688]    250
dtype: int64

In [33]:
# 类似的，在cut中我们可以自己指定百分比：
cats2 = pd.cut(data, [0, 0.1, 0.5, 0.9, 1.]) # 累进的百分比
cats2

[NaN, NaN, NaN, NaN, (0.5, 0.9], ..., NaN, (0.5, 0.9], (0.5, 0.9], (0.1, 0.5], NaN]
Length: 1000
Categories (4, interval[float64]): [(0.0, 0.1] < (0.1, 0.5] < (0.5, 0.9] < (0.9, 1.0]]

In [34]:
pd.value_counts(cats2)

(0.1, 0.5]    180
(0.5, 0.9]    129
(0.0, 0.1]     32
(0.9, 1.0]     26
dtype: int64

In [35]:
# --------------6 Detecting and Filtering Outliers（检测和过滤异常值）----------------

In [36]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.012039,-0.026797,0.026527,0.031155
std,1.045484,0.981771,0.975626,1.012426
min,-3.313562,-2.956280,-3.017711,-3.515004
25%,-0.731918,-0.701275,-0.655999,-0.644990
50%,0.036800,-0.021866,0.012262,0.034069
75%,0.658151,0.600705,0.683066,0.701527
max,3.106621,3.045988,3.055542,3.059983


In [37]:
data.head()

,0,1,2,3
0,1.723039,0.518173,0.299874,-0.779423
1,0.127692,-2.257052,1.321052,-0.288449
2,-0.023695,0.146502,0.148723,-0.452801
3,0.438927,-1.159759,0.733524,0.508614
4,0.982702,-0.380047,-0.507586,-1.563432


In [38]:
# 我们想要找一个列中，绝对值大于3的数字
col = data[2]
col.head()

0    0.299874
1    1.321052
2    0.148723
3    0.733524
4   -0.507586
Name: 2, dtype: float64

In [39]:
col[np.abs(col) > 3]

127    3.055542
336   -3.017711
Name: 2, dtype: float64

In [40]:
# 选中所有绝对值大于3的行，可以用any方法在一个boolean DataFrame上
data[(np.abs(data) > 3).any(1)] # any中axis=1表示column,any只要有一个符合就行

,0,1,2,3
127,-0.392718,-0.853981,3.055542,-0.715992
309,-1.301761,-0.563295,0.170415,-3.515004
336,-1.484528,-0.438905,-3.017711,1.437619
437,-3.218036,0.066818,-1.811449,1.760168
443,-3.313562,1.619784,-1.170699,0.282145
452,-3.041045,-0.056078,0.589839,0.037728
518,-3.212959,0.393672,0.195931,0.732822
596,2.656944,-0.221847,1.718034,3.059983
663,0.497917,3.045988,-0.174718,1.660096
722,3.106621,-0.176155,0.556243,0.745303


In [41]:
data[(np.abs(data) > 3).all(1)]  # all需要全部符合

,0,1,2,3


In [42]:
# -----------7 Permutation and Random Sampling（排列和随机采样）---------------

In [43]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [44]:
# 通过需要排列的轴的长度，调用permutation，产生一个表示新顺序的数组
sampler = np.random.permutation(5)
sampler

array([0, 3, 1, 2, 4])

In [45]:
df.take(sampler) #数组能被用在基于iloc上的indexing或take函数

,0,1,2,3
0,0,1,2,3
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11
4,16,17,18,19


In [46]:
# 为了选中一个随机的子集，而且没有代替功能(既不影响原来的值，返回一个新的series或DataFrame)，可以用sample方法：
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
3,12,13,14,15


In [48]:
# 给sample中设定replace=True
# 生成的样本带有替代功能
choices = pd.Series([5, 7, -1, 6, 4])

draws = choices.sample(n=10, replace=True)

draws

1    7
0    5
3    6
4    4
2   -1
0    5
2   -1
3    6
3    6
3    6
dtype: int64

In [49]:
# ------------8 Computing Indicator/Dummy Variables（计算指标/哑变量）--------------

In [50]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [51]:
# 把一个categorical variable(类别变量)变为一个dummy or indicator matrix（哑变量或指标矩阵）
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [52]:
# 要给column加一个prefix， 可以用data.get_dummies里的prefix参数来实现
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [54]:
# 这样就等于进行了one-hot转化

df_with_dummy = df[['data1']].join(dummies)  
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [55]:
# 一个有用的诀窍是把get_dummies和离散函数（比如cut）结合起来
np.random.seed(12345)
values = np.random.rand(10)
values

array([ 0.92961609,  0.31637555,  0.18391881,  0.20456028,  0.56772503,
        0.5955447 ,  0.96451452,  0.6531771 ,  0.74890664,  0.65356987])

In [56]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1.]
pd.cut(values, bins)

[(0.8, 1.0], (0.2, 0.4], (0.0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.4, 0.6], (0.8, 1.0], (0.6, 0.8], (0.6, 0.8], (0.6, 0.8]]
Categories (5, interval[float64]): [(0.0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1.0]]

In [57]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0
